In [1]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import time
from tqdm import tqdm_notebook
from sklearn import tree
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import datasets

CLASS_NAMES=['bass', 'brass', 'flute', 'guitar', 
             'keyboard', 'mallet', 'organ', 'reed', 
             'string', 'synth_lead', 'vocal']

SOURCE_NAMES=['acoustic', 'electronic', 'synthetic']
DATA_GROUPS=['test', 'valid', 'train']
FEATURE_LIST=['y_harmonic', 'y_percussive', 'chroma_cens', 'mfcc','mel_spec', 'spec_contrast']

def getDataset(dataGroup, source, target, other):#, instrument, source):
    
    new_dir='Dataset/nsynth-'+dataGroup+'/audio/'     #set the audio directory (test, train, etc)
    dataframe_raw = pd.read_json(path_or_buf='Dataset/nsynth-'+dataGroup+'/examples.json', orient='index') #read all instruments from examples.json
    dataframe_specific = dataframe_raw.loc[(dataframe_raw['instrument_family_str'] == target) | (dataframe_raw['instrument_family_str'] == other)]           #narrow down by family (strings, etc)
    dataframe_specific = dataframe_specific.loc[dataframe_specific['instrument_source_str'] == source]     #narrow down by source (acoustic, etc)

   
    Y = dataframe_specific.instrument_family_str.replace(to_replace=[other, target], value=[0, 1])
    filenames = dataframe_specific.index.tolist()     #get filenames from our dataframe, put into list
    
    dictionary = {}
    for file in tqdm_notebook(filenames):             #for all files in filenames. Also,  tqdm is a loading bar
       # print(new_dir)
       # print(file)
       # print('.wav')
       # print(new_dir+file+'.wav')
        features = feature_extract((new_dir+file+'.wav')) #specify directory, file, then add .wav. we will perform feature_extract with the file
        dictionary[file] = features                       #make dictionary using file as rows - features as columns
    featureDf = pd.DataFrame.from_dict(dictionary, orient='index', #turn into dataframe
                                       columns=FEATURE_LIST)
    featureFinal = pd.concat([dataframe_specific, featureDf], axis=1, sort=False)

    featureFinal['targetValue'] = Y
    return featureFinal #returns dataframe of features

def feature_extract(file):
    y, sr = librosa.load(file, sr=None)
    
    hop_length = 512
    
    # Separate harmonics and percussives into two waveforms
    y_harmonic, y_percussive = librosa.effects.hpss(y)    

    #Mel Spectrogram
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, 
                                                 fmax = 8000)
    #Mel-Frequency Cepstral Coefficients (MFCC) features from the raw signal
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)
    
    #Spectral Contrast
    spec_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    
    chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr)
    
    y_harmonic = np.mean(y_harmonic)
    y_percussive = np.mean(y_percussive)
    mel_spec = np.mean(mel_spec)
    mfcc = np.mean(mfcc, axis =1)
    chroma_cens = np.mean(chroma_cens)
    spec_contrast = np.mean(spec_contrast)
    
    return [y_harmonic, y_percussive, chroma_cens, mfcc, mel_spec, 
            spec_contrast]

In [2]:
def runSVM():
    dataGroup = DATA_GROUPS[0]    #SET IF YOU WANT TEST, TRAIN, OR VALID (IF YOU HAVE IT)
    instrumentTarget = CLASS_NAMES[8]   #SET THE INSTRUMENT YOU WANT
    source = SOURCE_NAMES[0]      #SET ACOUSTIC, ELECTRONIC, SYNTHETIC
    instrumentOther = CLASS_NAMES[4]

    test = getDataset(dataGroup, source, instrumentTarget, instrumentOther)   #GETS DATAFRAME WITH FEATURES EXTRACTED BY AVERY

    testX = test[FEATURE_LIST[0]].values.reshape(-1, 1)
    testY = test['targetValue'].values.reshape(-1, 1)

    print(test.targetValue)

    
    
    
    dataGroup = DATA_GROUPS[2]    #SET IF YOU WANT TEST, TRAIN, OR VALID (IF YOU HAVE IT)
    instrumentTarget = CLASS_NAMES[8]   #SET THE INSTRUMENT YOU WANT
    source = SOURCE_NAMES[0]      #SET ACOUSTIC, ELECTRONIC, SYNTHETIC
    instrumentOther = CLASS_NAMES[4]

    train = getDataset(dataGroup, source, instrumentTarget, instrumentOther)
    trainX = train[FEATURE_LIST[0]].array.reshape(-1, 1)
    trainY = train['targetValue'].array.reshape(-1, 1)

    clf = svm.SVC(kernel='linear')

    clf.fit(trainX, trainY)

    prediction = clf.predict(testX)
    accuracy = metrics.accuracy_score(testY, prediction)

    print("Accuracy: ", accuracy)
    return(prediction, accuracy)

In [3]:
#prediction,accuracy = runSVM()
#dataGroup = DATA_GROUPS[0]    #SET IF YOU WANT TEST, TRAIN, OR VALID (IF YOU HAVE IT)
#instrumentTarget = CLASS_NAMES[8]   #SET THE INSTRUMENT YOU WANT
#source = SOURCE_NAMES[0]      #SET ACOUSTIC, ELECTRONIC, SYNTHETIC
#instrumentOther = CLASS_NAMES[4]

#test = getDataset(dataGroup, source, instrumentTarget, instrumentOther)   #GETS DATAFRAME WITH FEATURES EXTRACTED BY AVERY

   

In [4]:
#dataGroup = DATA_GROUPS[2]    #SET IF YOU WANT TEST, TRAIN, OR VALID (IF YOU HAVE IT)
#instrumentTarget = CLASS_NAMES[8]   #SET THE INSTRUMENT YOU WANT
#source = SOURCE_NAMES[0]      #SET ACOUSTIC, ELECTRONIC, SYNTHETIC
#instrumentOther = CLASS_NAMES[4]

#train = getDataset(dataGroup, source, instrumentTarget, instrumentOther)

In [5]:
#prediction,accuracy = runSVM()
dataGroup = DATA_GROUPS[0]    #SET IF YOU WANT TEST, TRAIN, OR VALID (IF YOU HAVE IT)
instrumentTarget = CLASS_NAMES[8]   #SET THE INSTRUMENT YOU WANT
source = SOURCE_NAMES[0]      #SET ACOUSTIC, ELECTRONIC, SYNTHETIC
instrumentOther = CLASS_NAMES[4]

test = pd.read_csv("test.csv")

dataGroup = DATA_GROUPS[2]    #SET IF YOU WANT TEST, TRAIN, OR VALID (IF YOU HAVE IT)
instrumentTarget = CLASS_NAMES[8]   #SET THE INSTRUMENT YOU WANT
source = SOURCE_NAMES[0]      #SET ACOUSTIC, ELECTRONIC, SYNTHETIC
instrumentOther = CLASS_NAMES[4]

train = pd.read_csv("train.csv")

In [17]:
train = train.sample(frac=1)
test = test.sample(frac=1)

for x in range(1): 
    #print(FEATURE_LIST[x])
    #testX = test.loc[:,[FEATURE_LIST[x]]]
    #testX = test[FEATURE_LIST[x]].values
    testX = test.drop(['targetValue'], axis=1)
    testX = test[['y_harmonic','y_percussive', 'chroma_cens','mfcc', 'mel_spec', 'spec_contrast']]
    testX = testX.to_numpy()
    testY = test['targetValue'].values
    #test[FEATURE_LIST[x]].values.reshape(-1, 1)
    #testY = test['targetValue'].values.reshape(-1, 1)

    #testX = testX.ravel()
    #testY = testY.ravel()
    
    #trainX = train.loc[:,[FEATURE_LIST[x]]]
    trainX = train[['y_harmonic', 'y_percussive','chroma_cens', 'mfcc', 'mel_spec', 'spec_contrast']]
    trainX = trainX.to_numpy()
    #trainX = train[FEATURE_LIST[x]].values
    trainY = train['targetValue'].values
    #trainX = train[FEATURE_LIST[x]].values.reshape(-1, 1)
    #trainY = train['targetValue'].values.reshape(-1, 1)

    #trainX = trainX.ravel()
    #trainY = trainY.ravel()

    #print(trainX.values)
    
    print(trainX)
    print(trainY)
    #bdf = datasets.load_breast_cancer()
    
    #print(bdf.data)
    
    #trainX, testX, trainY, testY = train_test_split(bdf.data, bdf.target, test_size=0.3,random_state=109)
    
    #print(trainX)
    #print(trainY)
    #print(testX)
    #print(testY)
    
    clf = svm.SVC(kernel='linear')#kernel='rbf', C=100, gamma=10)

    print(trainX.shape)
    print(trainY.shape)
    print(trainX)
    print(trainY)
    
    print(testX.shape)
    
    clf.fit(trainX, trainY)

    prediction = clf.predict(testX)
    accuracy = metrics.accuracy_score(testY, prediction)
    
    print(prediction)

    print("Accuracy ",FEATURE_LIST[x],": ", accuracy)


[[-3.07166083e-05 -9.06469577e-05  1.77589028e-01 -1.81866417e+01
   7.27748299e+00  2.59701879e+01]
 [ 5.75740842e-06 -3.56249757e-06  1.24513993e-01 -4.30868492e+01
   5.40208340e-01  2.30403341e+01]
 [ 1.57342996e-07 -5.17807564e-07  1.95047449e-01 -3.19856873e+01
   2.33462071e+00  2.79821943e+01]
 ...
 [ 5.67790400e-03  1.06937710e-04  2.15127750e-01 -2.26923027e+01
   2.00748849e+00  3.02138538e+01]
 [-1.59582356e-07  8.30484339e-07  1.81291939e-01 -1.21194401e+01
   6.78115034e+00  2.32143341e+01]
 [ 3.19001963e-08 -2.03483765e-07  1.78430171e-01 -3.47276802e+01
   4.19028187e+00  2.61501757e+01]]
[1 1 1 ... 0 1 1]
(27458, 6)
(27458,)
[[-3.07166083e-05 -9.06469577e-05  1.77589028e-01 -1.81866417e+01
   7.27748299e+00  2.59701879e+01]
 [ 5.75740842e-06 -3.56249757e-06  1.24513993e-01 -4.30868492e+01
   5.40208340e-01  2.30403341e+01]
 [ 1.57342996e-07 -5.17807564e-07  1.95047449e-01 -3.19856873e+01
   2.33462071e+00  2.79821943e+01]
 ...
 [ 5.67790400e-03  1.06937710e-04  2.15127